In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
from langchain.document_loaders import UnstructuredExcelLoader

# 엑셀 파일 로드 및 문서로 변환
xlsx_path = 'test용사전엑셀.xlsx'
xlsx_loader = UnstructuredExcelLoader(xlsx_path, mode="elements")
docs = xlsx_loader.load()

# Embeddings 및 문서 분할 설정
embeddings = OpenAIEmbeddings()
semantic_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
semantic_chunks = semantic_chunker.create_documents([d.page_content for d in docs])


C:\Users\201\AppData\Local\Temp\ipykernel_15336\438272867.py:15: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [3]:
from pymongo import MongoClient
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain.embeddings import OpenAIEmbeddings 
import os

MONGODB_ATLAS_CLUSTER_URI = os.getenv("MONGODB_ATLAS_CLUSTER_URI")
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

db_name = "test"
collection_name = "col"

collection = client[db_name][collection_name]

# 4. Vector Search 초기화
vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents=docs,  
    embedding=embeddings,      
    collection=collection       
)

In [4]:
# 번역을 위한 PromptTemplate 설정
translation_prompt = PromptTemplate(
    input_variables=["retrieved_text", "query"],
    template="""
다음은 한국어 텍스트와 관련된 참고 자료입니다:
참고 자료: {retrieved_text}

위 자료를 기반으로, 아래 한국어 문장을 영어로 번역해주세요.
문장: {query}
번역:
"""
)

trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

translation_chain = LLMChain(
    llm=trans_llm, 
    prompt=translation_prompt
    )

retriever = vector_search.as_retriever()


query = "산독끼는 바나나와 꼬라니를 좋아해."
retrieved_docs = retriever.get_relevant_documents(query)

# 검색 결과를 결합하여 참고 자료로 활용
retrieved_text = " ".join([doc.page_content for doc in retrieved_docs])

result = translation_chain.run({"retrieved_text": retrieved_text, "query": query})
print("번역 결과:", result)

C:\Users\201\AppData\Local\Temp\ipykernel_15336\864608299.py:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
C:\Users\201\AppData\Local\Temp\ipykernel_15336\864608299.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  translation_chain = LLMChain(
C:\Users\201\AppData\Local\Temp\ipykernel_15336\864608299.py:25: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_doc

번역 결과: The mountain rat likes bananas and corn.
